In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/home/mcamara/taxi_demand_predictor")

In [3]:
import src.config as config

In [4]:
from datetime import datetime

In [5]:
import pandas as pd

In [6]:
current_date = pd.to_datetime(datetime.utcnow()).floor('H')

In [7]:
from src.feature_store_api import get_feature_group

In [8]:
predictions_fg = get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTIONS)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/24729
Connected. Call `.close()` to terminate connection gracefully.


In [9]:
actuals_fg = get_feature_group(name=config.FEATURE_GROUP_NAME)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/24729
Connected. Call `.close()` to terminate connection gracefully.


In [10]:
from datetime import timedelta

In [11]:
query = predictions_fg.select_all().join(actuals_fg.select_all(), 
                                         on=['pickup_hour', 'pickup_location_id']).filter(predictions_fg.pickup_hour >= current_date - timedelta(days=30))

In [12]:
from src.feature_store_api import get_feature_store

In [13]:
feature_store = get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/24729
Connected. Call `.close()` to terminate connection gracefully.


In [14]:
try:
    # create feature view as it does not exist yet\n",
    feature_store.create_feature_view( 
    name=config.FEATURE_VIEW_MONITORING, 
    version=1, 
    query=query 
    ) 
except: 
    print('Feature view already existed. Skip creation.') 
     

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/24729/fs/24649/fv/predictions_vs_actuals_for_monitoring_feature_view/version/1


In [15]:
predictions_and_actuals_fv = feature_store.get_feature_view(name=config.FEATURE_VIEW_MONITORING, 
                                                            version=1) 

In [16]:
preds_and_actuals = predictions_and_actuals_fv.get_batch_data(start_time=(current_date - timedelta(days=30)),
                                                              end_time=current_date)
   

2023-03-28 17:47:26,578 INFO: USE `taxi_demand_mc_featurestore`
2023-03-28 17:47:27,066 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`pickup_location_id` `pickup_location_id`, `fg1`.`predicted_demand` `predicted_demand`, `fg1`.`pickup_hour` `pickup_hour`, `fg1`.`pickup_location_id` `join_pk_pickup_location_id`, `fg1`.`pickup_hour` `join_pk_pickup_hour`, `fg1`.`pickup_hour` `join_evt_pickup_hour`, `fg0`.`rides` `rides`, RANK() OVER (PARTITION BY `fg1`.`pickup_location_id`, `fg1`.`pickup_hour`, `fg1`.`pickup_hour` ORDER BY `fg0`.`pickup_hour` DESC) pit_rank_hopsworks
FROM `taxi_demand_mc_featurestore`.`model_predictions_feature_group_1` `fg1`
INNER JOIN `taxi_demand_mc_featurestore`.`time_series_hourly_feature_group_1` `fg0` ON `fg1`.`pickup_location_id` = `fg0`.`pickup_location_id` AND `fg1`.`pickup_hour` = `fg0`.`pickup_hour` AND `fg1`.`pickup_hour` >= `fg0`.`pickup_hour`
WHERE `fg1`.`pickup_hour` >= TIMESTAMP '2023-02-26 15:00:00.000' AND `fg1`.`pickup_hour` >= TIMESTAMP '2023

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/24729/featurestores/24649/featureview/predictions_vs_actuals_for_monitoring_feature_view/version/1/trainingdatasets/version/1). Server response: 
HTTP code: 404, HTTP reason: Not Found, error code: 270012, error msg: Training dataset wasn't found., user msg: FeatureView name: predictions_vs_actuals_for_monitoring_feature_view, version: 1, td version: 1

In [ ]:
preds_and_actuals = preds_and_actuals[preds_and_actuals.pickup_hour.between(current_date - timedelta(days=30),
                                                                            current_date)]

In [ ]:
df = predictions_and_actuals

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mae = mean_absolute_error(df['rides'], df['predicted_demand'])
print(f'{mae=:.2f}')